# **Proyecto Final:** 
**Alumnos:** Aroca Aguilar, Antonio José y Díaz Ramírez, José Antonio  
**Correos:** antonioaroca@correo.ugr.es y joseadr@correo.ugr.es  
**Asignatura:** Visión por Computador  
**Profesor:** Nicolás Pérez de la Blanca Capilla  
**Curso:** 2020/21

#Instalación e import necesarios

In [1]:
!pip install -q keras #instalamos keras en google colab

In [ ]:
#importamos algunos modulos necesarios
import numpy as np 
import matplotlib.pyplot as plt
import math 
import os
import seaborn as sns
import pandas as pd

#importamos OpenCV para el tratamiento de imagenes
import cv2 as cv 
from google.colab.patches import cv2_imshow #importamos el parche para poder usar imshow de opencv en colab

#importamos lo necesario de keras
import keras
import keras.utils as np_utils
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Reshape, Lambda, Input, LeakyReLU, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
import keras.backend as K

#importamos el drive
from google.colab import drive
drive.mount("/content/drive") #montamos el sistema de archivos

#Funciones implementadas

In [3]:
"""
funcion para cargar las imagenes de del path en un diccionario considerando como 
clave el tipo de habitacion que es, la ciudad a la que pertence y el identificador
asociado a esa habitacion concreta.
Entrada: la ruta de la carpeta principal donde estan contenidas las carpetas de las ims
Salida: un diccionario con la clave ya mencionada y como valor para cada clave una
        lista con las imagenes que pertenecen a la misma clase(clave)
"""
def cargarImagenes(path):

  #obtenemos todos los directorios donde estan almacenadas las imagenes
  ruta = path
  directorios = os.listdir(ruta)
  directorios.remove('_DS_Store')

  diccionario = {}
  for carpeta in directorios: 

    #recorremos cada directorio obteniendo el nombre de las imagenes que contiene
    ruta = path + '/' + carpeta
    imagenes = os.listdir(ruta)
    
    for imagen in imagenes:
      
      #cargamos cada imagen obtenemos su ciudad y el codigo asociado a la habitacion
      ruta = path + '/' + carpeta + '/' + imagen
      im = load_img(ruta,target_size=(256,256))
      im = img_to_array(im)
      ciudad = imagen.split('_')[0]
      codigo = imagen.split('_')[1]
      clave = carpeta + '_' + ciudad + '_' + codigo

      #la metemos en el diccionario teniendo en cuenta si ya se han cargado mas imagenes de su misma clase previamente
      if clave in diccionario.keys():
        images = diccionario[clave]
        images.append(im)
        diccionario[clave] = images
      else:
        diccionario[clave] = [im]

  return diccionario

"""
funcion para cargar las imagenes de la base de datos AirBnb
Entrada: -
Salida: un diccionario con las imagenes del trainning y otro con las del test
"""
def cargarImagenesAirBnb():

  diccionarioTrain = cargarImagenes("./drive/My Drive/imagenes/Airbnb Data/Training Data")
  diccionarioTest = cargarImagenes("./drive/My Drive/imagenes/Airbnb Data/Test Data")

  return diccionarioTrain, diccionarioTest

"""
funcion para eliminar las clases que solamente tienen un ejemplo porque no van a
tener casos positivos
Entrada: el diccionario con todas las imagenes
Salida: el diccionario de entradas una vez eliminadas las entradas no validas
"""
def eliminarEjemplosNoValidos(diccionario):

  clavesEliminar = []

  for clave in diccionario:
    if len(diccionario[clave]) == 1:
      clavesEliminar.append(clave)
  
  for key in clavesEliminar:
    del diccionario[key]
  
  return diccionario

"""
funcion para transformar los datos de las imagenes disponibles en un diccionario
en dos listas
Entrada: el diccionario con todas las imagenes
Salida: una lista con todas las imagenes del diccionario y otra lista con las clases
        a las que pertenece cada imagen. Ambas listas siguen el mismo orden de modo 
        que clases[i] es la clase de imagenes[i]
"""
def leerImagenes(diccionario):

  clases =[]
  imagenes = []

  for clave in diccionario:
    for img in range(len(diccionario[clave])):
      clases.append(clave)
      imagenes.append(diccionario[clave][img])
  
  return imagenes, clases

"""
funcion para obtener los conjuntos de entrenamiento y test y sus distintas etiquetas
en el formato correcto. (variacion de la funcion proporcionada por los profesores de 
la asignatura para la realizacion de la practica 2)
Entrada: el diccionario con las imagenes del trainning y el diccionario con las 
        imagenes del test
Salida: una lista con las imagenes del conjunto training barajadas, una matriz en 
        la que cada fila es una sucesion de 0s menos un 1 en la columna que represente
        la clase a la que pertenece la imagen correspondiente, lo equivalente para 
        el conjunto test
"""
def cargarDatos(diccionario,diccionario_test):
  
  #convertimos los diccionarios en listas con la funcion leerImagenes
  train, train_clases = leerImagenes(diccionario)
  test, test_clases = leerImagenes(diccionario_test)
  
  # Pasamos los vectores de las clases a matrices. Para ello, primero pasamos las clases a números enteros
  clases_posibles = np.unique(np.copy(train_clases))
  train_clases = [i for i in range(len(clases_posibles)) for j in range(len(train_clases)) if train_clases[j] == clases_posibles[i]]
  
  clases_posibles_test = np.unique(np.copy(test_clases))
  test_clases = [i for i in range(len(clases_posibles_test)) for j in range(len(test_clases)) if test_clases[j] == clases_posibles_test[i]]
    
  # Después, usamos la función to_categorical()
  train_clases = np_utils.to_categorical(train_clases, 274)
  test_clases = np_utils.to_categorical(test_clases, 111)
  
  # Barajar los datos
  train_perm = np.random.permutation(len(train))
  training = []
  for i in range(len(train_perm)):
    training.append(train[train_perm[i]])
    
  test_perm = np.random.permutation(len(test))
  test_final = []
  for i in range(len(test_perm)):
    test_final.append(test[test_perm[i]])

  #ordenamos las clases segun la permutacion de imagenes anterior
  train_clases = train_clases[train_perm]
  test_clases = test_clases[test_perm]

  #antes de devolver lo convertimos a arrays de numpy
  training = np.array(training)
  test_final = np.array(test_final)

  return training, train_clases, test_final, test_clases

"""
funcion para obtener el subconjunto de validacion como un subconjunto del train
Entrada: el conjunto train y sus etiquetas y la probabilidad de que cada elemento
        pertenezca al conjunto train
Salida: los conjuntos de train y validacion con sus respectivas etiquetas
"""
def formarValidation(xTrain,yTrain,probTrain):

  #creamos las variables que necesitamos
  indicesTrain = []
  indicesValidation = []
  x_train = []
  y_train =[]
  x_val = [] 
  y_val = []

  #recorremos todos los elementos de train seleccionando cuales perteneceran a train y cuales a validation
  for i in range(len(xTrain)):
    if np.random.rand() < probTrain:
      indicesTrain.append(i)
    else:
      indicesValidation.append(i)

  #creamos el nuevo conjunto train y sus etiquetas
  for i in range(len(indicesTrain)):
    x_train.append(xTrain[indicesTrain[i]])
    y_train.append(yTrain[indicesTrain[i]])

  #creamos el nuevo conjunto validation y sus etiquetas
  for i in range(len(indicesValidation)):
    x_val.append(xTrain[indicesValidation[i]])
    y_val.append(yTrain[indicesValidation[i]])

  return x_train, y_train, x_val, y_val

"""
funcion para mostrar la evolucion del accuracy y el loss del conjunto train y del 
validacion a lo largo de las epocas (funcion proporcionada por los profesores de 
la asignatura para la realizacion de la practica 2)
Entrada: el histograma dado por el aprendizaje
Salida: -
"""
def mostrarEvolucion(hist):

  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training loss', 'Validation loss'])
  plt.show()

  acc = hist.history['acc']
  val_acc = hist.history['val_acc']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training accuracy', 'Validation accuracy'])
  plt.show()

"""
funcion para hacer parejas de imagenes (primera version con la misma cantidad de 
parejas positivas que negativas)
Entrada: las imagenes en forma de lista y sus etiquetas de forma categorica, tal y
        como nos las proporciona la funcion cargarDatos
Salida: una lista de parejas de imagenes y una lista de etiquetas podiendo ser estas
        un 1 en caso de que ambas imagenes de la pareja pertenezca a la misma clase 
        y un 0 en caso contrario
"""
def make_pairs_v1(images, labels):

  parejas = []
  clasesParejas = []
  
  #obtenemos una lista con los indices de las imagenes que tienen cada clase
  numClases = len(labels[0])
  indice = []
  for i in range(0,len(labels)):
    for j in range(0,len(labels[i])):
      if (labels[i][j]==1):
        indice.append(j)
  indice = np.array(indice)
  idx = [np.where(indice == i)[0] for i in range(0, numClases)]

  #recorremos todas las imagenes formando con cada una de ellas una pareja positiva y una negativa
  for im in range(len(images)):

    #guardamos la imagen actual y su etiqueta
    imagenActual = images[im]
    label = indice[im]
    
    #escogemos aleatoriamente otra imagen de su misma clase para formar pareja positiva
    num_indices = np.where(indice == label)[0]
    salir = True
    while salir :
      idxB = np.random.choice(idx[label])
      if not idxB == im:
        salir=False
    posImage = images[idxB]
    #añadimos la pareja y su clase
    parejas.append([imagenActual, posImage])
    clasesParejas.append([1])
	  
    #escogemos aleatoriamente una imagen de su otra clase distinta para formar pareja negativa
    negIdx = np.where(indice != label)[0]
    idxC = np.random.choice(negIdx)
    negImage = images[idxC]
		#añadimos la pareja y su clase
    parejas.append([imagenActual, negImage])
    clasesParejas.append([0])
	
  return parejas, clasesParejas

"""
funcion para hacer parejas de imagenes (segunda version con el doble de parejas 
negativas que positivas)
Entrada: las imagenes en forma de lista y sus etiquetas de forma categorica, tal y
        como nos las proporciona la funcion cargarDatos
Salida: una lista de parejas de imagenes y una lista de etiquetas podiendo ser estas
        un 1 en caso de que ambas imagenes de la pareja pertenezca a la misma clase 
        y un 0 en caso contrario
"""
def make_pairs_v2(images, labels):

  parejas = []
  clasesParejas = []
  
  #obtenemos una lista con los indices de las imagenes que tienen cada clase
  numClases = len(labels[0])
  indice = []
  for i in range(0,len(labels)):
    for j in range(0,len(labels[i])):
      if (labels[i][j]==1):
        indice.append(j)
  indice = np.array(indice)
  idx = [np.where(indice == i)[0] for i in range(0, numClases)]

  #recorremos todas las imagenes formando con cada una de ellas una pareja positiva y una negativa
  for im in range(len(images)):

    #guardamos la imagen actual y su etiqueta
    imagenActual = images[im]
    label = indice[im]
    
    #escogemos aleatoriamente otra imagen de su misma clase para formar pareja positiva
    num_indices = np.where(indice == label)[0]
    salir = True
    while salir :
      idxB = np.random.choice(idx[label])
      if not idxB == im:
        salir=False
    posImage = images[idxB]
    #añadimos la pareja y su clase
    parejas.append([imagenActual, posImage])
    clasesParejas.append([1])
	  
    #escogemos aleatoriamente una imagen de su otra clase distinta para formar pareja negativa
    negIdx = np.where(indice != label)[0]
    idxC = np.random.choice(negIdx)
    negImage = images[idxC]
		#añadimos la pareja y su clase
    parejas.append([imagenActual, negImage])
    clasesParejas.append([0])

    #escogemos aleatoriamente otra imagen de su otra clase distinta para formar pareja negativa
    negIdx = np.where(indice != label)[0]
    salir = True
    while salir:
      idxD = np.random.choice(negIdx)
      if not idxC == idxD:
        salir = False
    negImage = images[idxD]
		#añadimos la pareja y su clase
    parejas.append([imagenActual, negImage])
    clasesParejas.append([0])
	
  return parejas, clasesParejas

"""
funcion para hacer parejas de imagenes (tercera version con el mismo numero de parejas
negativas que positivas, pero con el doble que en la version 1)
Entrada: las imagenes en forma de lista y sus etiquetas de forma categorica, tal y
        como nos las proporciona la funcion cargarDatos
Salida: una lista de parejas de imagenes y una lista de etiquetas podiendo ser estas
        un 1 en caso de que ambas imagenes de la pareja pertenezca a la misma clase 
        y un 0 en caso contrario
"""
def make_pairs_v3(images, labels):

  parejas = []
  clasesParejas = []
  
  #obtenemos una lista con los indices de las imagenes que tienen cada clase
  numClases = len(labels[0])
  indice = []
  for i in range(0,len(labels)):
    for j in range(0,len(labels[i])):
      if (labels[i][j]==1):
        indice.append(j)
  indice = np.array(indice)
  idx = [np.where(indice == i)[0] for i in range(0, numClases)]

  #recorremos todas las imagenes formando con cada una de ellas una pareja positiva y una negativa
  for im in range(len(images)):

    #guardamos la imagen actual y su etiqueta
    imagenActual = images[im]
    label = indice[im]
    
    #escogemos aleatoriamente otra imagen de su misma clase para formar pareja positiva
    num_indices = np.where(indice == label)[0]
    salir = True
    while salir :
      idxB = np.random.choice(idx[label])
      if not idxB == im:
        salir=False
    posImage = images[idxB]
    #añadimos la pareja y su clase
    parejas.append([imagenActual, posImage])
    clasesParejas.append([1])

    #escogemos aleatoriamente otra imagen de su misma clase para formar pareja positiva
    salir = True
    while salir :
      idxB = np.random.choice(idx[label])
      if not idxB == im:
        salir=False
    posImage = images[idxB]
		#añadimos la pareja y su clase
    parejas.append([imagenActual, posImage])
    clasesParejas.append([1])
	  
    #escogemos aleatoriamente una imagen de su otra clase distinta para formar pareja negativa
    negIdx = np.where(indice != label)[0]
    idxC = np.random.choice(negIdx)
    negImage = images[idxC]
		#añadimos la pareja y su clase
    parejas.append([imagenActual, negImage])
    clasesParejas.append([0])

    #escogemos aleatoriamente otra imagen de su otra clase distinta para formar pareja negativa
    negIdx = np.where(indice != label)[0]
    salir = True
    while salir:
      idxD = np.random.choice(negIdx)
      if not idxC == idxD:
        salir = False
    negImage = images[idxD]
		#añadimos la pareja y su clase
    parejas.append([imagenActual, negImage])
    clasesParejas.append([0])
	
  return parejas, clasesParejas

"""
funcion para hacer parejas de imagenes, hace tantas positivas como se pueda sin 
repetir parejas y tantas negativas como positivas tengamos
Entrada: un diccionario en el que las claves son las clases y contiene una lista
        con las imagenes que pertenezcan a esa clase
Salida: una lista de parejas de imagenes y una lista de etiquetas podiendo ser estas
        un 1 en caso de que ambas imagenes de la pareja pertenezca a la misma clase 
        y un 0 en caso contrario
"""
def hacerParejas(dict):

  parejas = []
  clases = []

  #para cada calse
  for clave in dict:

    imagenes = dict[clave] #obtenemos las imagenes de esa clase
    
    for i in range(len(imagenes)-1): #para cada una de estas imagenes

      cantidadParejas = 0

      #formamos las parejas con el resto de imagenes de esa clase sin que haya repeticion
      for j in range(i+1,len(imagenes)):
        parejas.append([imagenes[i],imagenes[j]])
        clases.append(1)
        cantidadParejas += 1
      
      #para cada una de las parejas formadas anteriormente formamos tambien una negativa aleatoria
      for j in range(cantidadParejas):
        while True:
          aleatorio = np.random.randint(0,len(dict.keys()))
          if not list(dict.keys())[aleatorio] == clave:
            im = dict[list(dict.keys())[aleatorio]]
            aleatorio = np.random.randint(0,len(im))
            parejas.append([imagenes[i],im[aleatorio]])
            clases.append(0)
            break
  
  #hacemos una permutacion para desordenarlas
  orden = np.random.permutation(len(parejas))
  parejasFinal = []
  clasesFinal = []
  for i in orden:
    parejasFinal.append(parejas[i])
    clasesFinal.append(clases[i])
  
  return parejasFinal,clasesFinal

"""
funcion para calcular la distancia euclidea entre dos vectores de caracteristicas
(funcion obtenida en la web:
https://www.pyimagesearch.com/2020/11/30/siamese-networks-with-keras-tensorflow-and-deep-learning/)
Entrada: los vectores de caracteristicas 
Salida: la distancia entre ellos 
"""
def euclidean_distance(vectors):
	
	(featsA, featsB) = vectors #obtenemos los dos vectores
	sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True) #suma de distancias cuadradas

	return K.sqrt(K.maximum(sumSquared, K.epsilon()))
 
"""
funcion para obtener la matriz de confusion de los resultados obtenidos
Entrada: los resultados obtenidos y los resultadso correctos
Salida: -
"""
def generarMatrizConfusion(bienIguales, malIguales, bienDistintas, malDistintas):

  matrizConfusion = [[bienDistintas,
                     malDistintas],
                     [malIguales,
                     bienIguales]]
  
  datos = pd.DataFrame(matrizConfusion)
  sns.heatmap(datos, center=0, cmap='Blues_r', annot=True, fmt='.3f')

"""
funcion para completar, compilar y entrenar un modelo
Entrada: el tamaño de las imagenes y los conjuntos de entrenamiento y validacion
Salida: el modelo y la historia del entrenamiento
"""
def completarCompilarEntrenarModelo(model,tamImagenes,par_x_train_final,par_y_train_final,par_x_val,par_y_val):

  #completamos el modelo
  imgA = Input(tamImagenes)
  imgB = Input(tamImagenes)
  featureExtractor = model
  featsA = featureExtractor(imgA)
  featsB = featureExtractor(imgB)

  distance = Lambda(euclidean_distance)([featsA, featsB])
  outputs = Dense(1, activation="sigmoid")(distance)
  model = Model(inputs=[imgA, imgB], outputs=outputs)

  #compilamos y declaramos el optimizador
  optimizador = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
  model.compile(loss=keras.losses.binary_crossentropy, optimizer=optimizador, metrics=['acc'])

  #compilamos el modelo
  history = model.fit([par_x_train_final[:,0], par_x_train_final[:,1]], par_y_train_final[:],validation_data=([par_x_val[:,0], par_x_val[:,1]], par_y_val[:]),
	batch_size=batch_size, epochs=epochs)
  
  return model, history

"""
funcion para obtener las predicciones de un modelo sobre unas parejas de imagenes dadas
Entrada: el modelo entrenado y el conjunto de test con sus etiquetas
Salida: -
"""
def predecir(model,par_x_test,par_y_test):
  bien1 = 0
  mal1 = 0
  mal2=0
  bien2=0
  total = 0
  # loop over all image pairs
  for i in range(len(par_x_test)):  
    imageA = par_x_test[i,0]
    imageB = par_x_test[i,1]

    origA = imageA.copy()
    origB = imageB.copy()
    # add channel a dimension to both the images
    imageA = np.expand_dims(imageA, axis=-1)
    imageB = np.expand_dims(imageB, axis=-1)
    # add a batch dimension to both images
    imageA = np.expand_dims(imageA, axis=0)
    imageB = np.expand_dims(imageB, axis=0)
    # scale the pixel values to the range of [0, 1]
    #imageA = imageA / 255.0
    #imageB = imageB / 255.0
    #print(imageB.shape)
    # use our siamese model to make predictions on the image pair,
    # indicating whether or not the images belong to the same class
    preds = model.predict([imageA, imageB])
    proba = preds[0][0]
    if (proba > 0.4 ):
      if (par_y_test[i]==1):
        bien1 +=1
      else:
        mal1 +=1
    else:
      if (par_y_test[i]==0):
        bien2 +=1
      else:
        mal2 +=1
  generarMatrizConfusion(bien1,mal1,bien2,mal2)
  total = bien1 + bien2 + mal1 + mal2
  porc = (bien1+bien2)*100 / total
  print("Ha acertado ", (bien1+bien2), " y ha fallado ", (mal1+mal2), " de un total de ", total, " siendo un ", porc )

# Modelos utilizados

In [4]:
"""
primer modelo implementado cogido de la siguiente web:
https://www.pyimagesearch.com/2020/11/30/siamese-networks-with-keras-tensorflow-and-deep-learning/
"""
def submodelo_prueba(inputShape):
  
	inputs = Input(inputShape) #especificamos el tamaño de las imagenes de entrada

	#definimos el conjunto de capas: CONV => RELU => POOL => DROPOUT
	x = Conv2D(64, (2, 2), padding="same", activation="relu")(inputs)
	x = MaxPooling2D(pool_size=(2, 2))(x)
	x = Dropout(0.3)(x)
	#definimos el segundo conjunto de capas: CONV => RELU => POOL => DROPOUT 
	x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
	x = MaxPooling2D(pool_size=2)(x)
	x = Dropout(0.3)(x)
  
  #preparamos el output final
	pooledOutput = GlobalAveragePooling2D()(x)
	outputs = Dense(274)(pooledOutput)
	model = Model(inputs, outputs) #contruimos el modelo

	return model

"""
segundo modelo escogido: modelo basado en la red resnet50 preentrenada con imagenet
al que se le ha añadido una capa dense al final con 274 (numero de clases que tenemos)
"""
def submodelo(inputShape):

  resnet50 = ResNet50(weights='imagenet', include_top=False, pooling="avg", input_shape=inputShape) #obtenemos resnet50

  #indicamos que no vuelva a entrenar las capas
  for layer in resnet50.layers:
    if (not isinstance(layer, keras.layers.BatchNormalization)): #keras tiene un bug con batchNormalization
      layer.trainable = False

  #preparamos el output final
  x = resnet50.output
  x = Dense(274)(x)

  model = Model(inputs = resnet50.input, outputs = x) #construimos el modelo

  return model

"""
tercer modelo escogido: modelo basado en la red resnet50 preentrenada con imagenet
al que se le ha añadido una capa dense al final con 274 (numero de clases que tenemos)
y haciendo fine-tunning sobre una serie de capas
"""
def submodelo_fine_tunning(inputShape,numero_capas):

  resnet50 = ResNet50(weights='imagenet', include_top=False, pooling="avg", input_shape=inputShape) #obtenemos resnet50

  #hacemos fine-tunning
  for layer in resnet50.layers[:numero_capas]: 
    if (not isinstance(layer, keras.layers.BatchNormalization)):  #keras tiene un bug con batchNormalization
      layer.trainable = False

  #preparamos el output final
  x = resnet50.output
  x = Dense(274)(x)

  model = Model(inputs = resnet50.input, outputs = x) #construimos el modelo

  return model

#Constantes utilizadas

In [5]:
batch_size = 32
num_classes = 25
epochs = 30

# Código principal

carga de imagenes

In [ ]:
#cargamos las imagenes
dicTrain, dicTest = cargarImagenesAirBnb()
dicTrain = eliminarEjemplosNoValidos(dicTrain)
dicTest = eliminarEjemplosNoValidos(dicTest)
print('imagenes cargadas correctamente')

primera version: submodelo_prueba y parejas formadas con make_pairs_v1

In [ ]:
#generamos las parejas de imagenes
x_train, y_train, x_test, y_test = cargarDatos(dicTrain,dicTest)
par_x_train , par_y_train = make_pairs_v1(x_train,y_train)
par_x_test, par_y_test = make_pairs_v1(x_test,y_test)
print('parejas creadas correctamente')

#obtenemos el modelo
modelo = submodelo_prueba((256,256,3))
modelo.summary()
print('modelo obtenido con exito')

#formamos el conjunto de validacion y entrenamos el modelo
par_x_train_final, par_y_train_final, par_x_val, par_y_val = formarValidation(par_x_train,par_y_train,0.8 )
par_x_train_final = np.asarray(par_x_train_final)
par_y_train_final = np.asarray(par_y_train_final)
par_x_val = np.asarray(par_x_val)
par_y_val = np.asarray(par_y_val)
print('conjunto validation obtenido correctamente')

model, hist = completarCompilarEntrenarModelo(modelo,(256,256,3),par_x_train_final,par_y_train_final,par_x_val,par_y_val)
print('entrenamiento completado')
mostrarEvolucion(hist)

#realizamos las predicciones
par_x_test = np.asarray(par_x_test)
par_y_test = np.asarray(par_y_test)
predecir(model,par_x_test,par_y_test)

segunda version: submodelo y parejas formadas con make_pairs_v1

In [ ]:
#generamos las parejas de imagenes
x_train, y_train, x_test, y_test = cargarDatos(dicTrain,dicTest)
par_x_train , par_y_train = make_pairs_v1(x_train,y_train)
par_x_test, par_y_test = make_pairs_v1(x_test,y_test)
print('parejas creadas correctamente')

#obtenemos el modelo
modelo = submodelo((256,256,3))
modelo.summary()
print('modelo obtenido con exito')

#formamos el conjunto de validacion y entrenamos el modelo
par_x_train_final, par_y_train_final, par_x_val, par_y_val = formarValidation(par_x_train,par_y_train,0.8 )
par_x_train_final = np.asarray(par_x_train_final)
par_y_train_final = np.asarray(par_y_train_final)
par_x_val = np.asarray(par_x_val)
par_y_val = np.asarray(par_y_val)
print('conjunto validation obtenido correctamente')

model, hist = completarCompilarEntrenarModelo(modelo,(256,256,3),par_x_train_final,par_y_train_final,par_x_val,par_y_val)
print('entrenamiento completado')
mostrarEvolucion(hist)

#realizamos las predicciones
par_x_test = np.asarray(par_x_test)
par_y_test = np.asarray(par_y_test)
predecir(model,par_x_test,par_y_test)

tercera version: submodelo y parejas formadas con make_pairs_v2

In [ ]:
#generamos las parejas de imagenes
x_train, y_train, x_test, y_test = cargarDatos(dicTrain,dicTest)
par_x_train , par_y_train = make_pairs_v2(x_train,y_train)
par_x_test, par_y_test = make_pairs_v2(x_test,y_test)
print('parejas creadas correctamente')

#obtenemos el modelo
modelo = submodelo((256,256,3))
modelo.summary()
print('modelo obtenido con exito')

#formamos el conjunto de validacion y entrenamos el modelo
par_x_train_final, par_y_train_final, par_x_val, par_y_val = formarValidation(par_x_train,par_y_train,0.8 )
par_x_train_final = np.asarray(par_x_train_final)
par_y_train_final = np.asarray(par_y_train_final)
par_x_val = np.asarray(par_x_val)
par_y_val = np.asarray(par_y_val)
print('conjunto validation obtenido correctamente')

model, hist = completarCompilarEntrenarModelo(modelo,(256,256,3),par_x_train_final,par_y_train_final,par_x_val,par_y_val)
print('entrenamiento completado')
mostrarEvolucion(hist)

#realizamos las predicciones
par_x_test = np.asarray(par_x_test)
par_y_test = np.asarray(par_y_test)
predecir(model,par_x_test,par_y_test)

cuarta version: submodelo y parejas formadas con make_pairs_v3

In [ ]:
#generamos las parejas de imagenes
x_train, y_train, x_test, y_test = cargarDatos(dicTrain,dicTest)
par_x_train , par_y_train = make_pairs_v3(x_train,y_train)
par_x_test, par_y_test = make_pairs_v3(x_test,y_test)
print('parejas creadas correctamente')

#obtenemos el modelo
modelo = submodelo((256,256,3))
modelo.summary()
print('modelo obtenido con exito')

#formamos el conjunto de validacion y entrenamos el modelo
par_x_train_final, par_y_train_final, par_x_val, par_y_val = formarValidation(par_x_train,par_y_train,0.8 )
par_x_train_final = np.asarray(par_x_train_final)
par_y_train_final = np.asarray(par_y_train_final)
par_x_val = np.asarray(par_x_val)
par_y_val = np.asarray(par_y_val)
print('conjunto validation obtenido correctamente')

model, hist = completarCompilarEntrenarModelo(modelo,(256,256,3),par_x_train_final,par_y_train_final,par_x_val,par_y_val)
print('entrenamiento completado')
mostrarEvolucion(hist)

#realizamos las predicciones
par_x_test = np.asarray(par_x_test)
par_y_test = np.asarray(par_y_test)
predecir(model,par_x_test,par_y_test)

quinta version: submodelo y parejas formadas con hacerParejas

In [ ]:
#generamos las parejas de imagenes
par_x_train, par_y_train = hacerParejas(dicTrain)
par_x_test, par_y_test = hacerParejas(dicTest)
print('parejas creadas correctamente')

#obtenemos el modelo
modelo = submodelo((256,256,3))
modelo.summary()
print('modelo obtenido con exito')

#formamos el conjunto de validacion y entrenamos el modelo
par_x_train_final, par_y_train_final, par_x_val, par_y_val = formarValidation(par_x_train,par_y_train,0.8 )
par_x_train_final = np.asarray(par_x_train_final)
par_y_train_final = np.asarray(par_y_train_final)
par_x_val = np.asarray(par_x_val)
par_y_val = np.asarray(par_y_val)
print('conjunto validation obtenido correctamente')

model, hist = completarCompilarEntrenarModelo(modelo,(256,256,3),par_x_train_final,par_y_train_final,par_x_val,par_y_val)
print('entrenamiento completado')
mostrarEvolucion(hist)

#realizamos las predicciones
par_x_test = np.asarray(par_x_test)
par_y_test = np.asarray(par_y_test)
predecir(model,par_x_test,par_y_test)

sexta version: submodelo_fine_tunning de todas las capas y parejas formadas con hacerParejas

In [ ]:
#generamos las parejas de imagenes
par_x_train, par_y_train = hacerParejas(dicTrain)
par_x_test, par_y_test = hacerParejas(dicTest)
print('parejas creadas correctamente')

#obtenemos el modelo
modelo = submodelo_fine_tunning((256,256,3),0)
modelo.summary()
print('modelo obtenido con exito')

#formamos el conjunto de validacion y entrenamos el modelo
par_x_train_final, par_y_train_final, par_x_val, par_y_val = formarValidation(par_x_train,par_y_train,0.8 )
par_x_train_final = np.asarray(par_x_train_final)
par_y_train_final = np.asarray(par_y_train_final)
par_x_val = np.asarray(par_x_val)
par_y_val = np.asarray(par_y_val)
print('conjunto validation obtenido correctamente')

model, hist = completarCompilarEntrenarModelo(modelo,(256,256,3),par_x_train_final,par_y_train_final,par_x_val,par_y_val)
print('entrenamiento completado')
mostrarEvolucion(hist)

#realizamos las predicciones
par_x_test = np.asarray(par_x_test)
par_y_test = np.asarray(par_y_test)
predecir(model,par_x_test,par_y_test)

septima version: submodelo_fine_tunning con el 20% de las capas reentrenadas y parejas formadas con hacerParejas

In [ ]:
#generamos las parejas de imagenes
par_x_train, par_y_train = hacerParejas(dicTrain)
par_x_test, par_y_test = hacerParejas(dicTest)
print('parejas creadas correctamente')

#obtenemos el modelo
modelo = submodelo_fine_tunning((256,256,3),10)
modelo.summary()
print('modelo obtenido con exito')

#formamos el conjunto de validacion y entrenamos el modelo
par_x_train_final, par_y_train_final, par_x_val, par_y_val = formarValidation(par_x_train,par_y_train,0.8 )
par_x_train_final = np.asarray(par_x_train_final)
par_y_train_final = np.asarray(par_y_train_final)
par_x_val = np.asarray(par_x_val)
par_y_val = np.asarray(par_y_val)
print('conjunto validation obtenido correctamente')

model, hist = completarCompilarEntrenarModelo(modelo,(256,256,3),par_x_train_final,par_y_train_final,par_x_val,par_y_val)
print('entrenamiento completado')
mostrarEvolucion(hist)

#realizamos las predicciones
par_x_test = np.asarray(par_x_test)
par_y_test = np.asarray(par_y_test)
predecir(model,par_x_test,par_y_test)

#Bibliografia:  
  

* https://stackoverflow.com/questions/43977463/valueerror-could-not-broadcast-input-array-from-shape-224-224-3-into-shape-2

* https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/load_img

*  https://en.wikipedia.org/wiki/Cropping_(image)
* https://jkjung-avt.github.io/keras-image-cropping/
* https://www.pyimagesearch.com/2020/11/30/siamese-networks-with-keras-tensorflow-and-deep-learning/
* https://www.pyimagesearch.com/2021/01/18/contrastive-loss-for-siamese-networks-with-keras-and-tensorflow/
* http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
* https://stats.stackexchange.com/questions/388859/is-it-possible-to-give-variable-sized-images-as-input-to-a-convolutional-neural
* https://medium.com/@prabhnoor0212/siamese-network-keras-31a3a8f37d04

* https://keras.io/api/optimizers/adam/